## Phi-4 RAG with Azure AI Search

এই নোটবুকটি দেখায় কীভাবে Phi-4-mini এবং Phi-4-multimodal ব্যবহার করে Retrieval Augmented Generation (RAG) করা যায় Azure AI Search এর সাথে। এটি একক-মোডালিটি (শুধুমাত্র টেক্সট) এবং বহু-মোডালিটি (টেক্সট এবং ছবি) উভয় পরিস্থিতি কভার করে।

**প্রয়োজনীয়তা:**
*   Azure AI Search ভেক্টর ইনডেক্স (একটি তৈরি করতে [এই নির্দেশনাগুলি](https://learn.microsoft.com/azure/search/search-get-started-portal-import-vectors?tabs=sample-data-storage%2Cmodel-aoai%2Cconnect-data-storage) অনুসরণ করুন)
*   Phi-4-mini অথবা Phi-4-multimodal এন্ডপয়েন্ট Azure AI Foundry-তে ডিপ্লয় করা


In [ ]:
! pip install azure-ai-inference azure-search-documents

### শুধুমাত্র টেক্সট-ভিত্তিক RAG ব্যবহার করে Phi-4-mini

এই অংশে দেখানো হয়েছে কীভাবে Phi-4-mini-কে একটি চ্যাট সম্পূর্ণতার মডেল হিসেবে RAG-এর জন্য ব্যবহার করা যায়, শুধুমাত্র টেক্সট ইনপুট হিসেবে। এটি Azure AI Foundry Inference এবং Azure AI Search-এ সংযোগ স্থাপন, প্রাসঙ্গিক ডকুমেন্টগুলি পুনরুদ্ধার, এবং পুনরুদ্ধারকৃত প্রসঙ্গ ব্যবহার করে একটি উত্তর তৈরি করার প্রক্রিয়া অন্তর্ভুক্ত করে।


In [ ]:
# Step 1: Connect to Azure AI Foundry Inference & Azure AI Search
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

chat_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], # Phi-4-mini endpoint
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL"]),
)

search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

# Step 2: Retrieve relevant documents from Azure AI Search
def retrieve_documents(query: str, top: int = 10):
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=top, fields="text_vector")
    results = search_client.search(search_text=query,vector_queries=[vector_query],select=["content"],top=top)
    return [doc["content"] for doc in results]

# Step 3: Generate answer using RAG!
def generate_rag_response(query: str):
    docs = retrieve_documents(query)
    context = "\n---\n".join(docs)
    prompt = f"""You are a helpful assistant. Use only the following context to answer the question. If the answer isn't in the context, say 'I don't know'.
    Context: {context} Question: {query} Answer:"""
    response = chat_client.complete(messages=[UserMessage(content=prompt)])
    return response.choices[0].message.content

# Example usage
user_query = "What is the capital of France?"
answer = generate_rag_response(user_query)
print(f"Q: {user_query}\nA: {answer}")


### মাল্টি-মোডাল RAG Phi-4-multimodal এর সাথে

এই অংশে দেখানো হয়েছে কীভাবে Phi-4-multimodal কে RAG এর জন্য একটি চ্যাট সম্পূর্ণ মডেল হিসেবে ব্যবহার করা যায়, যেখানে টেক্সট এবং ইমেজ ইনপুট উভয়ই অন্তর্ভুক্ত করা হয়েছে। এটি Azure AI Inference এবং Azure AI Search এর সাথে সংযোগ স্থাপন, প্রাসঙ্গিক ডকুমেন্ট পুনরুদ্ধার, এবং একটি মাল্টিমোডাল প্রতিক্রিয়া তৈরি করার প্রক্রিয়া কভার করে।

**নোট:** আপনি মাল্টি-ভেক্টর কোয়েরি করতে পারেন যদি আপনার Azure AI Search ইনডেক্সে `text_vector` এবং `image_vector` ফিল্ড উভয়ই থাকে।


In [ ]:
# Step 1: Connect to Azure AI Inference & Azure AI Search
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery


chat_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], #Phi-4-multimodal serverless endpoint
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL"]),
)

search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

# Step 2: Retrieve relevant documents from Azure AI Search
def retrieve_documents(query: str, top: int = 3):
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=top, fields="text_vector")
    results = search_client.search(search_text=query, vector_queries=[vector_query], select=["content"], top=top)    
    return [doc["content"] for doc in results]

## Example for Muli-modal search if you have a text_vector AND image_vector field in your vector_index
## NOTE, image vectorization is in preview at the time of writing this code, please use azure-search-documents pypi version >11.6.0b6 
# def retrieve_documents_multimodal(query: str, image_url: str, top: int = 3):
#     text_vector_query = VectorizableTextQuery(
#         text=query,
#         k_nearest_neighbors=top,
#         fields="text_vector",
#         weight=0.5  # Adjust weight as needed
#     )
#     image_vector_query = VectorizableImageUrlQuery(
#         url=image_url,
#         k_nearest_neighbors=top,
#         fields="image_vector",
#         weight=0.5  # Adjust weight as needed
#     )

#     results = search_client.search(
#         search_text=query,  
#         vector_queries=[text_vector_query, image_vector_query],
#         select=["content"],
#         top=top
#     )
#     return [doc["content"] for doc in results]


# Step 3: Generate a multimodal RAG-based answer using retrieved text and an image input
def generate_multimodal_rag_response(query: str, image_url: str):
    # Retrieve text context from search
    docs = retrieve_documents(query)
    context = "\n---\n".join(docs)

    # Build a prompt that combines the retrieved context with the user query
    prompt = f"""You are a helpful assistant. Use only the following context to answer the question. If the answer isn't in the context, say 'I don't know'.
    Context: {context} Question: {query} Answer:"""
    # Create a chat request that includes both text and image input
    response = chat_client.complete(
        messages=[
            SystemMessage(content="You are a helpful assistant that can process both text and images."),
            UserMessage(
                content=[
                    TextContentItem(text=prompt),
                    ImageContentItem(image_url=ImageUrl(url=image_url, detail=ImageDetailLevel.HIGH)),
                ]
            ),
        ]
    )
    return response.choices[0].message.content

# Example usage
user_query = "Can you search for similar items to this shoe?"
sample_image_url = "https://images.unsplash.com/photo-1542291026-7eec264c27ff?q=80&w=1770&auto=format&fit=crop&ixlib=rb-4.0.3"
answer = generate_multimodal_rag_response(user_query, sample_image_url)
print(f"Q: {user_query}\nA: {answer}")


---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসাধ্য সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
